In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio

In [ ]:
# get the images and their corresponding labels

train_dataset_path = Path("/kaggle/input/alzheimer-diseases-3-class/3_cls/train")
training_images = list(train_dataset_path.glob(r'**/*.png'))
training_labels = list(map(lambda x: x.parents[0].stem, training_images))

#Test Dataset
test_dataset_path = Path("/kaggle/input/alzheimer-diseases-3-class/3_cls/test")
test_images = list(test_dataset_path.glob(r'**/*.png'))
test_labels = list(map(lambda x: x.parents[0].stem, test_images))

In [ ]:
training_images = pd.Series(training_images, name="Images").astype(str)
training_labels = pd.Series(training_labels, name="Labels").astype(str)

In [ ]:
test_images = pd.Series(test_images, name="Images").astype(str)
test_labels = pd.Series(test_labels, name="Labels").astype(str)

In [ ]:
training_data = pd.concat([training_images, training_labels], axis = 1)
training_data = training_data.sample(frac = 1, random_state = 42).reset_index(drop = True)
training_data

In [ ]:
test_data = pd.concat([test_images, test_labels], axis = 1)
test_data = test_data.sample(frac = 1, random_state = 42).reset_index(drop = True)
test_data

In [ ]:
 #Viewing some of the images in the dataframe
fig, axes = plt.subplots(2, 2, figsize=(8,8))
for i, ax in enumerate(axes.flat):
    ax.imshow(imageio.imread(training_data.Images[i]))
    ax.set_title(training_data.Labels[i])
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import mobilenet

train_generator_mobile_net = ImageDataGenerator(
    preprocessing_function = mobilenet.preprocess_input,
    validation_split = 0.1
)

test_generator_mobile_net = ImageDataGenerator(
    preprocessing_function = mobilenet.preprocess_input
)

In [ ]:
from sklearn.model_selection import train_test_split

train_df = training_data.copy()
test_df = test_data.copy()

train = train_generator_mobile_net.flow_from_dataframe(
    dataframe=train_df,
    x_col="Images",
    y_col="Labels",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training',
    rotation_range=32,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    sheer_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5,1.5],
    fill_mode="nearest"
)

validation = train_generator_mobile_net.flow_from_dataframe(
    dataframe=train_df,
    x_col="Images",
    y_col="Labels",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation',
    rotation_range=32,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    sheer_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5,1.5],
    fill_mode="nearest"
)

test = test_generator_mobile_net.flow_from_dataframe(
    dataframe=test_df,
    x_col="Images",
    y_col="Labels",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    rotation_range=32,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shuffle=False
)

In [ ]:
#using MobileNet

from tensorflow.keras.applications import mobilenet

mobilenet_ = mobilenet.MobileNet(
    input_shape=(224, 224, 3),
    include_top=False,
    alpha=1.0,
    weights='imagenet',
    pooling='avg'
)

mobilenet_.trainable = False

In [ ]:
CHECKPOINTS = Path("./checkpoints")
CHECKPOINTS.mkdir(exist_ok=True)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout


In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

# building the Predictor layers
x = Dense(128, activation='relu')(mobilenet_.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)

outputs = Dense(3, activation='softmax')(x)

mobilenet = Model(inputs=mobilenet_.inputs, outputs=outputs)

mobilenet.compile(
    optimizer=Adam(),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy()]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

with tf.device('/GPU:0'):
# training
    history = mobilenet.fit(
        train,
        validation_data = validation,
        batch_size = 32,
        epochs = 20,
        callbacks = [EarlyStopping(
            monitor="val_loss",
            patience=4,
            restore_best_weights=True
        ), ReduceLROnPlateau(patience=2)]
    )

In [ ]:
# Testing the model
results = mobilenet.evaluate(test)

In [ ]:
predictions = np.argmax(mobilenet.predict(test), axis=1)

In [ ]:
from sklearn.metrics import classification_report
labels = dict((v, k) for k, v in train.class_indices.items())
actual = list(test_df.Labels)
predictions = [labels[i] for i in predictions]
print(classification_report(actual, predictions))

In [ ]:
labels = ['CONTROL', 'AD', 'PD']

In [ ]:
from io import StringIO
import pandas as pd
# Get classification report as a string
report_str = classification_report(actual, predictions, target_names = labels)

# Read the classification report string into a Pandas DataFrame
cls_df = pd.read_csv(StringIO(report_str), sep='\s{2,}', engine='python', skipfooter=3)
cls_df

In [ ]:
from io import BytesIO
# Plot DataFrame as a table
fig, ax = plt.subplots(figsize=(10, 6))
ax.axis('off')  # Turn off the axis
table = ax.table(cellText=cls_df.values, colLabels=cls_df.columns, rowLabels=cls_df.index, loc='center', cellLoc='center', colColours=['#f2f2f2']*len(cls_df.columns))

# Save the figure as an image
image_stream = BytesIO()
plt.savefig(image_stream, format='png', bbox_inches='tight', pad_inches=0.5)
plt.close()  # Close the plot to avoid displaying it

# Display the image or save it to a file
# For example, to save it to a file:
image_stream.seek(0)
with open('mobileNet_cls_df.png', 'wb') as f:
    f.write(image_stream.read())

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cf = confusion_matrix(actual, predictions, normalize = "true")
plt.figure(figsize=(8, 6))
sns.heatmap(cf,cmap='Blues', annot=True, xticklabels = sorted(set(actual)), yticklabels = sorted(set(actual)))
plt.title('MobileNet Confusion Matrix')
plt.show()

In [ ]:
q = len(list(history.history['loss']))
plt.figure(figsize=(12, 6))
sns.lineplot(x = range(1, 1+q), y = history.history['categorical_accuracy'], label = 'Accuracy')
sns.lineplot(x = range(1, 1+q), y = history.history['loss'], label = 'Loss')
plt.xlabel('#epochs')
plt.ylabel('Training')
plt.legend();

In [ ]:
pd.DataFrame(history.history)[['categorical_accuracy', 'val_categorical_accuracy']].plot()
plt.title("Accuracy")
plt.show()

In [ ]:
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
# Metrics 
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, precision_score, recall_score, f1_score

In [ ]:
print("precision: ",precision_score(actual, predictions, average = "weighted"))
print("recall: ",recall_score(actual, predictions, average = "weighted") )
print("f1_score: ",f1_score(actual, predictions, average = "weighted"))
print("accuracy_score: ", accuracy_score(actual, predictions))

In [ ]:
cmat = confusion_matrix(actual, predictions)
classwise_acc = cmat.diagonal()/cmat.sum(axis=1) * 100 
cls_acc = pd.DataFrame({'Class_Label':[labels[i] for i in range(3)], 'Accuracy': classwise_acc.tolist()}, columns = ['Class_Label', 'Accuracy'])
cls_acc.style.format({"Accuracy": "{:,.2f}",}).bar(subset=["Accuracy"], color='tomato')

In [ ]:
labels

In [ ]:
import matplotlib.pyplot as plt

# Data
metrics = ['Precision', 'Recall', 'F1 Score', 'Accuracy']
scores = [0.7136,0.7070,0.7043,0.7070]

# Plotting the bar graph
plt.bar(metrics, scores, color=['blue', 'green', 'orange', 'red'])

# Adding labels and title
plt.xlabel('Metrics')
plt.ylabel('Scores')
plt.title('Performance Metrics')

# Display the scores on top of the bars
for i, score in enumerate(scores):
    plt.text(i, score + 0.0001, f'{score:.4f}', ha='center')

# Show the plot
plt.show()

